In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

In [2]:
referate_dev_df = pd.read_json('./referate-dev.json')
referate_test_df = pd.read_json('./referate-test.json')
referate_train_df = pd.read_json('./referate-train.json')

In [3]:
referate_dev_df

,index,filename,text,category,grade
0,3245,Modurile Personale,MODURILE PERSONALE \nINDICATIVUL arată o acţiu...,romana,8.00
1,4579,Referat Baltagul - rezumat,Tehnologia cresterii cainilor de serviciu -ref...,romana,7.12
2,756,Referat Noapte de toamna - George Toparceanu,REVOLUTIA FRANCEZA si biserica \nDate crono...,romana,6.45
3,5315,Referat Ion Luca Caragiale - O scrisoare pierduta,Ion luca caragiale - O scrisoare pierdut ă ...,romana,5.19
4,1835,"Referat Bastinasii Americii, Dezvoltare Izolat...","Victor Babes (1854- 1926), medic roman, fiul ...",istorie,5.75
...,...,...,...,...,...
495,888,Mitocondriile,Mitocondriile „uzinele energetice celulare” \n...,biologie,10.00
496,4158,Pestii,Tema: \nObiectul: Zoologia Profesor: ...,biologie,8.00
497,2851,Succesul Si Insuccesul Scolar,la pedagogie \nTema: Succesul şi insuccesul...,psihologie,8.00
498,2641,Ion Crenga - Harap Alb,Ion Creanga Povestea lui Harap-Alb \nFara indo...,romana,9.00


In [4]:
referate_test_df

,index,filename,text,category
0,1420,"Referat Ultima noapte de dragoste, intaia noap...",Elemente de noutate in estetica romanului Rom...,romana
1,1549,Referat Stiluri de viata in Romania,AZTECI \nAztecii au fost membrii uneia dintre ...,psihologie
2,1265,Modernizare-turcia,Efortul de modernizare în Turcia în perioada i...,istorie
3,5291,Luceafarul - Comentariu Var-1,LUCEAFĂRUL Despre felul cum s-a născut marele ...,romana
4,791,Referat Scurta istorie a bombei,Bombele atomice lansate de catre Statele Unite...,istorie
...,...,...,...,...
495,3591,Morometi,Morometii \n-Personaje principale \n -Parin...,romana
496,5389,Padurea Spanzuratilor - Comentariu,Padurea spanzuratilor de Liviu Rebreanu \nTema...,romana
497,1496,Referat Marsul lui Radetzky,MARELE ZID CHINEZESC \n Constructie gi...,istorie
498,911,Referat Ciresarii - Cavalerii florii de cires,A Z U G A Valea Prahovei a ...,romana


In [5]:
referate_train_df

,index,filename,text,category,grade
0,3685,Referat Piercing-ul si adolescentii,Piercing-ul si tatuajele - vanitati adolescent...,psihologie,5.65
1,775,Gradinile Suspendate Ale Semiramidei,GRADINILE SUSPENDATE ALE SEMIRAMIDEI A doua mi...,istorie,9.00
2,1765,Referat Mihail Sadoveanu,MIHAIL SADOVEANU (1880-1961) \nMihail Sadovea...,romana,5.36
3,3691,Referat Eminescu,EMINESCU \nS-a nascut pe 15 ianuarie 1850 la ...,romana,5.35
4,3402,Moara,Moara cu noroc Ioan Slavici \nCaracterizarea l...,romana,10.00
...,...,...,...,...,...
4720,2241,America De Nord,Civilizaţiile precolumbiene Poziţia geografică...,geografie,9.00
4721,1334,Simulare Examen De Capacitate 2003,Simulare examen de capacitate 2003 ...,istorie,10.00
4722,1379,Referat Fanus Neagu - Dincolo de nisipuri,Asia cea mai mare dintre părţilelumii.(30% di...,romana,7.21
4723,3943,Referat Ion Barbu - Etapele creaţiei poetice,Ion Barbu - Etapele creaţiei poetice \nFenomen...,romana,5.19


In [6]:
copy_df_train = referate_train_df.copy(deep=False)
copy_df_test = referate_test_df.copy(deep=False)
copy_df_dev = referate_dev_df.copy(deep=False)

In [7]:
X_train = copy_df_train.drop('grade',axis=1)
y_train = copy_df_train['grade']
X_test = copy_df_test
X_dev = copy_df_dev.drop('grade',axis=1)
y_dev = copy_df_dev["grade"]

In [8]:
all_df = pd.concat([X_train, referate_test_df, X_dev])
all_df

,index,filename,text,category
0,3685,Referat Piercing-ul si adolescentii,Piercing-ul si tatuajele - vanitati adolescent...,psihologie
1,775,Gradinile Suspendate Ale Semiramidei,GRADINILE SUSPENDATE ALE SEMIRAMIDEI A doua mi...,istorie
2,1765,Referat Mihail Sadoveanu,MIHAIL SADOVEANU (1880-1961) \nMihail Sadovea...,romana
3,3691,Referat Eminescu,EMINESCU \nS-a nascut pe 15 ianuarie 1850 la ...,romana
4,3402,Moara,Moara cu noroc Ioan Slavici \nCaracterizarea l...,romana
...,...,...,...,...
495,888,Mitocondriile,Mitocondriile „uzinele energetice celulare” \n...,biologie
496,4158,Pestii,Tema: \nObiectul: Zoologia Profesor: ...,biologie
497,2851,Succesul Si Insuccesul Scolar,la pedagogie \nTema: Succesul şi insuccesul...,psihologie
498,2641,Ion Crenga - Harap Alb,Ion Creanga Povestea lui Harap-Alb \nFara indo...,romana


In [9]:
le = preprocessing.LabelEncoder()
le.fit(all_df['filename'])
X_train['filename'] = le.transform(X_train['filename'].astype(str))
X_test['filename'] = le.transform(X_test['filename'].astype(str))
X_dev['filename'] = le.transform(X_dev['filename'].astype(str))

In [10]:
le = preprocessing.LabelEncoder()
le.fit(all_df['text'].astype(str))
X_train['text'] = le.transform(X_train[['text']].astype(str))
X_test['text'] = le.transform(X_test['text'].astype(str))
X_dev['text'] = le.transform(X_dev['text'].astype(str))

c:\users\eu\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [11]:
le = preprocessing.LabelEncoder()
le.fit(all_df['category'].astype(str))
X_train['category'] = le.transform(X_train['category'].astype(str))
X_test['category'] = le.transform(X_test['category'].astype(str))
X_dev['category'] = le.transform(X_dev['category'].astype(str))

In [25]:
my_model = RandomForestRegressor(n_estimators = 1000, random_state = 0)
my_model.fit(X_train, y_train)
pred = my_model.predict(X_dev)
print(pred)

[9.133   5.56509 6.04095 5.91284 5.70277 8.785   6.77997 8.907   9.427
 9.0515  8.539   5.99921 6.12233 6.45216 9.1885  8.818   8.682   9.705
 5.45821 5.78945 6.04018 9.218   6.44897 8.835   8.952   5.73737 6.19408
 8.775   9.0015  6.2548  6.09423 9.366   6.49785 9.0735  8.934   5.7679
 8.96    9.147   5.81944 9.092   5.49151 5.93443 5.75655 5.86918 5.34942
 5.3892  8.9325  8.896   6.24607 5.87804 6.3593  5.56609 6.01885 8.9225
 8.8205  5.98221 8.847   6.32142 6.26548 8.916   6.01474 6.64404 5.66447
 8.866   9.141   9.37    8.8165  5.14543 5.88062 6.71148 8.988   5.74871
 9.2645  5.84513 5.44078 8.473   6.76521 9.175   5.64826 9.3505  6.25066
 9.134   9.0435  5.64087 6.0146  8.918   5.45496 8.922   6.16463 8.9675
 5.60778 6.06309 8.665   8.671   8.453   6.27711 9.164   8.967   9.1845
 5.6203  8.782   5.94435 8.8905  5.70253 8.846   5.40203 9.142   6.09574
 8.693   8.9955  8.7375  9.466   8.6565  5.88348 5.8391  5.94263 6.09599
 6.31092 5.92844 9.326   9.019   6.32313 6.21598 6.39851 5.

In [13]:

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_dev, pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_dev, pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_dev, pred)))

Mean Absolute Error: 0.7581566400000004
Mean Squared Error: 0.8097435377456006
Root Mean Squared Error: 0.8998575096900623


In [26]:
y_test = my_model.predict(X_test)
y_test = np.round(y_test, 2)

In [15]:
y_test

array([5.92, 5.49, 9.01, 8.98, 5.75, 8.81, 9.57, 9.07, 8.67, 9.1 , 6.96,
       9.02, 6.44, 8.89, 9.11, 8.87, 6.15, 6.16, 5.53, 6.03, 6.42, 5.7 ,
       5.69, 9.25, 5.81, 6.31, 5.94, 5.67, 8.79, 8.65, 9.35, 8.37, 6.2 ,
       6.11, 8.74, 6.53, 5.54, 9.37, 9.1 , 9.19, 5.82, 6.2 , 9.03, 9.02,
       8.77, 8.68, 6.54, 5.57, 6.2 , 8.98, 8.48, 9.61, 9.22, 8.75, 5.96,
       8.21, 9.09, 6.3 , 9.04, 5.88, 6.08, 6.81, 8.92, 5.93, 9.06, 9.33,
       8.94, 6.46, 5.62, 9.62, 8.96, 6.21, 9.28, 9.51, 5.54, 5.87, 8.67,
       8.8 , 8.3 , 6.5 , 6.02, 9.52, 6.13, 6.46, 5.77, 5.76, 6.51, 5.66,
       9.63, 9.19, 9.04, 8.94, 8.91, 6.99, 5.78, 5.91, 8.86, 9.15, 9.17,
       5.67, 6.14, 8.33, 9.6 , 6.59, 8.59, 7.05, 6.34, 9.15, 9.07, 8.82,
       8.37, 8.85, 8.99, 5.92, 6.05, 5.86, 9.09, 8.87, 8.95, 5.64, 8.9 ,
       6.05, 9.34, 8.6 , 6.17, 5.97, 9.17, 9.16, 9.37, 5.62, 9.04, 8.94,
       6.26, 9.13, 5.74, 6.08, 6.74, 6.42, 8.79, 8.55, 5.66, 8.98, 6.72,
       8.65, 5.88, 8.3 , 8.55, 6.46, 9.05, 5.66, 6.

In [16]:
referate_test_df['grades'] = y_test.tolist()
referate_test_df

,index,filename,text,category,grades
0,1420,"Referat Ultima noapte de dragoste, intaia noap...",Elemente de noutate in estetica romanului Rom...,romana,5.92
1,1549,Referat Stiluri de viata in Romania,AZTECI \nAztecii au fost membrii uneia dintre ...,psihologie,5.49
2,1265,Modernizare-turcia,Efortul de modernizare în Turcia în perioada i...,istorie,9.01
3,5291,Luceafarul - Comentariu Var-1,LUCEAFĂRUL Despre felul cum s-a născut marele ...,romana,8.98
4,791,Referat Scurta istorie a bombei,Bombele atomice lansate de catre Statele Unite...,istorie,5.75
...,...,...,...,...,...
495,3591,Morometi,Morometii \n-Personaje principale \n -Parin...,romana,8.93
496,5389,Padurea Spanzuratilor - Comentariu,Padurea spanzuratilor de Liviu Rebreanu \nTema...,romana,8.79
497,1496,Referat Marsul lui Radetzky,MARELE ZID CHINEZESC \n Constructie gi...,istorie,5.75
498,911,Referat Ciresarii - Cavalerii florii de cires,A Z U G A Valea Prahovei a ...,romana,5.84


In [27]:
new_X_train = pd.concat([X_train, X_dev])
new_y_train = pd.concat([y_train, y_dev])

In [28]:
my_model = RandomForestRegressor(n_estimators = 1000, random_state = 0)
my_model.fit(new_X_train, new_y_train)

pred = my_model.predict(X_dev)
print(pred)

[8.413   6.50184 6.30571 5.37214 5.75103 8.287   6.29886 9.599   9.193
 9.0475  8.89    5.56671 5.7742  6.02317 8.454   8.936   9.548   9.883
 5.77511 5.59566 5.81125 8.472   5.65997 8.339   9.645   5.55605 7.05522
 8.414   9.636   5.74907 5.59915 9.10919 6.02216 9.6675  9.592   5.9677
 9.625   8.425   5.66666 8.416   5.36716 5.62535 5.91936 5.36021 5.76508
 5.44351 8.3055  9.704   5.77857 5.55584 5.3212  5.47423 5.73026 9.475
 8.929   5.60954 9.562   6.05117 5.77222 8.2585  5.4936  5.91572 5.4824
 9.57    8.365   9.114   9.579   5.32602 5.5834  6.18194 9.645   5.57165
 9.0865  5.24547 5.39621 8.771   6.3845  9.0585  5.57776 9.1555  5.59717
 9.091   8.41    5.44729 5.5314  8.3185  5.66991 9.614   5.63297 9.629
 6.40615 5.99948 8.847   8.348   8.161   5.23939 8.401   9.057   9.1065
 5.94439 9.645   5.22639 8.378   5.72749 8.329   6.28492 9.693   5.509
 8.943   9.0125  8.8905  8.571   8.802   5.41605 6.91076 6.81901 6.07363
 5.85305 5.82952 9.138   8.535   7.67126 5.67617 7.27553 6.52257

In [19]:

print('Mean Absolute Error:', metrics.mean_absolute_error(y_dev, pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_dev, pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_dev, pred)))

Mean Absolute Error: 0.2764209999999979
Mean Squared Error: 0.10883021587079857
Root Mean Squared Error: 0.32989424952672114


In [29]:
y_test2 = my_model.predict(X_test)

print(y_test2)

[6.01594 5.4817  8.9905  8.95449 5.69859 8.9495  9.4985  9.142   8.8225
 9.0705  7.12699 9.041   6.41401 8.735   9.174   8.858   5.99512 6.14801
 5.47484 5.88746 6.25899 5.62089 5.68379 9.187   5.84637 6.37256 5.93556
 5.62682 8.853   8.606   9.4745  8.557   6.21295 6.07011 8.9245  6.31512
 5.59849 9.21201 9.198   9.237   5.70774 6.30611 9.0715  9.0285  8.731
 8.3395  6.58791 5.57749 6.01302 8.8805  8.58807 9.621   9.2305  8.8048
 6.00967 8.082   8.99899 6.19385 9.044   5.90812 5.8628  6.71601 9.048
 5.88998 9.08732 9.1795  8.9225  6.40442 5.56887 9.66    9.0173  6.20727
 9.365   9.649   5.57756 5.77219 9.122   8.7855  8.287   6.36951 6.08113
 9.447   6.0747  6.29042 5.76099 5.64433 6.40176 5.62471 9.6565  9.231
 9.14896 8.891   9.02    6.93322 5.77112 5.94278 8.8195  9.2545  9.306
 5.69864 6.13793 8.263   9.611   6.54324 8.4995  7.068   6.3726  9.081
 9.002   8.959   8.345   8.922   8.965   5.89145 5.99489 5.93235 8.901
 9.0245  8.866   5.68275 9.0245  5.91636 9.419   8.613   6.23727 

In [21]:
referate_test_df.to_json(r'./rezultate.json')

In [22]:
rezultat_df = pd.read_json('./rezultate.json')

In [23]:
rezultat_df

,index,filename,text,category,grades
0,1420,"Referat Ultima noapte de dragoste, intaia noap...",Elemente de noutate in estetica romanului Rom...,romana,5.92
1,1549,Referat Stiluri de viata in Romania,AZTECI \nAztecii au fost membrii uneia dintre ...,psihologie,5.49
2,1265,Modernizare-turcia,Efortul de modernizare în Turcia în perioada i...,istorie,9.01
3,5291,Luceafarul - Comentariu Var-1,LUCEAFĂRUL Despre felul cum s-a născut marele ...,romana,8.98
4,791,Referat Scurta istorie a bombei,Bombele atomice lansate de catre Statele Unite...,istorie,5.75
...,...,...,...,...,...
495,3591,Morometi,Morometii \n-Personaje principale \n -Parin...,romana,8.93
496,5389,Padurea Spanzuratilor - Comentariu,Padurea spanzuratilor de Liviu Rebreanu \nTema...,romana,8.79
497,1496,Referat Marsul lui Radetzky,MARELE ZID CHINEZESC \n Constructie gi...,istorie,5.75
498,911,Referat Ciresarii - Cavalerii florii de cires,A Z U G A Valea Prahovei a ...,romana,5.84
